In [31]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    db,
    MilvusClient
)
import clip
import torch
import numpy as np
from PIL import Image
from torch.nn.functional import cosine_similarity

In [2]:
host = "localhost"
port = 19530 # Mapping for 19530 (default Milvus port)

# Connect to Milvus
client = connections.connect("default", host=host, port=port)

# Check if the connection is established
print("Is Milvus connected:", connections.has_connection("default"))

# Optional: List collections to confirm the connection
from pymilvus import utility
print("Collections:", utility.list_collections())

Is Milvus connected: True
Collections: ['images', 'product_collection', 'products']


In [3]:
database = db.create_database("Building_Designs")

In [4]:
db.using_database("Building_Designs")

In [5]:
fields = [
    FieldSchema(name="Building_id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="Image_Name", dtype=DataType.VARCHAR, max_length = 1000),
    FieldSchema(name="building_real_image", dtype=DataType.FLOAT_VECTOR, dim=512),  
    FieldSchema(name="generated_building_image", dtype=DataType.FLOAT_VECTOR, dim=512),
    FieldSchema(name="building_sketch", dtype=DataType.FLOAT_VECTOR, dim=512)
]

In [6]:
design_schema = CollectionSchema(fields, description="Products collection")
buildings_collection = Collection(name = 'designs', schema=design_schema)

In [7]:
collections = utility.list_collections()
print(collections)

['designs']


In [8]:
index_params = {
    "metric_type": "COSINE",
    "index_type": "IVF_FLAT",
    "params": {"nlist": 256}
}

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

clip_model, preprocess = clip.load("ViT-B/16", device=device)

In [45]:
def generate_image_embedding(preprocess, image):
    # Check if the input is a path (string) or a NumPy array
    if isinstance(image, str):
        # Load the image from the file path
        image_pil = Image.open(image)
    elif isinstance(image, np.ndarray):
        # Convert the NumPy array to a PIL Image
        image_pil = Image.fromarray(image)
    else:
        raise ValueError("The input must be a file path or a NumPy array")

    # Preprocess the image using the provided preprocessing function
    image_tensor = preprocess(image_pil).unsqueeze(0).to(device)

    with torch.no_grad():
        image_embedding = clip_model.encode_image(image_tensor)
        image_embedding /= image_embedding.norm(dim=-1, keepdim=True)

    return image_embedding.cpu().flatten()

In [67]:
embedding = generate_image_embedding(preprocess, r'D:\VS Code Folders\Pix2Pix_Buildings\sketches_images\building01_front_day_sketch.jpg')
embedding2 = generate_image_embedding(preprocess, r'D:\VS Code Folders\Pix2Pix_Buildings\sketches_images\building04_right_cloud_sketch.jpg')


In [68]:
import torch.nn.functional as F

# Calculate cosine similarity using PyTorch
similarity = F.cosine_similarity(embedding.unsqueeze(0), embedding2.unsqueeze(0))
print("Cosine Similarity:", similarity.item())


Cosine Similarity: 0.8912643790245056
